# Envibert Cased

In [1]:
from src.resources import hparams
from torch import nn
import torch

/home/tuyendv/anaconda3/envs/ner-capu/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from importlib.machinery import SourceFileLoader
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import RobertaModel
from fairseq.models.roberta import XLMRModel
from src.model.bilstm import BiLSTM
from src.resources import hparams
from torch import nn
import os

2022-08-21 20:51:18 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [3]:
bert_type = "xlmr"

In [4]:
if bert_type =="envibert_cased":
    print("use: envibert")
    tokenizer = SourceFileLoader("envibert.tokenizer", 
            os.path.join(hparams.toknizer_path,'envibert_tokenizer.py')).load_module().RobertaTokenizer(hparams.toknizer_path)
    bert = RobertaModel.from_pretrained('nguyenvulebinh/envibert',cache_dir=hparams.pretrained_envibert_cased)
elif bert_type =="envibert_uncased":
    print("use: envibert_uncased")
    tokenizer = SourceFileLoader("envibert.tokenizer", 
            os.path.join(hparams.pretrained_envibert_uncased,'envibert_tokenizer.py')).load_module().RobertaTokenizer(hparams.pretrained_envibert_uncased)
    bert = XLMRModel.from_pretrained(hparams.pretrained_envibert_uncased, checkpoint_file='model.pt')
elif bert_type =="xlmr":
    print("use: xlm roberta")
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
    bert = XLMRobertaModel.from_pretrained("xlm-roberta-base")
    

use: xlm roberta


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
class bert_bilstm(nn.Module):
    def __init__(self, nb_label, cuda, bert, drop_rate, hidden_dim_lstm, hidden_dim_bert):
        super().__init__()
        self.bert = bert
        self.bilstm = BiLSTM(
            cuda=cuda, 
            embedding_dim=hidden_dim_bert, 
            hidden_dim=hidden_dim_lstm
            )
        self.dropout = nn.Dropout(drop_rate)
        self.linear = nn.Linear(hidden_dim_lstm, nb_label).to(cuda)

    def forward(self, input_ids, input_masks):
        output = self.bert(input_ids=input_ids, 
                        attention_mask = input_masks)
        sequence_output, _ = output[0], output[1]
        
        sequence_output = self.dropout(sequence_output)
        sequence_output = self.bilstm(sequence_output)
        output = self.linear(sequence_output)

        return output

In [6]:
model = bert_bilstm(
        cuda="cuda:0",
        nb_label=hparams.nb_labels, 
        bert=bert,
        drop_rate=hparams.drop_rate,
        hidden_dim_bert=768,
        hidden_dim_lstm=hparams.hidden_dim_lstm).to("cuda:0")

In [7]:
tmp = torch.load("/home/tuyendv/projects/restore-text/checkpoint/xlmr/checkpoint_xlmr_2.pt")

In [10]:
res = {}
for key, value in tmp.items():
    res[key.replace("module.","")] = value
model.load_state_dict(res)

<All keys matched successfully>

In [ ]:
torch.save(model.state_dict(),"/home/tuyendv/projects/serving-model/restore-model-xlmr/checkpoint/restore_model_xlmr.pt")

In [29]:
model = torch.load("envibert_cased.pt", map_location="cpu").to("cpu")
model.cuda="cpu"
model.bilstm.cuda="cpu"

In [30]:
input_ids = torch.tensor([[1,2,3,4]])
label_ids = torch.tensor([[1,1,1,1]])

In [31]:
model(input_ids ,label_ids)

tensor([[[ 3.4678, -1.3652,  2.9402,  0.9296,  0.9075, -1.0552, -0.6884,
          -0.7643, -1.6479],
         [ 5.6365, -0.8979,  3.7384,  0.9441, -1.7830, -1.7870,  1.0364,
          -1.9206, -0.6754],
         [-3.3766, -2.8819, -3.0972,  2.8551, -3.8219,  0.1179,  2.5970,
          -4.4640, -0.8132],
         [-4.7732, -3.0532, -3.4493,  3.8218,  0.0276,  0.3232, -0.1796,
          -3.0593, -3.1788]]], grad_fn=<AddBackward0>)

# Envibert uncased

In [ ]:
from src.resources import hparams
from torch import nn
import torch

In [ ]:
from importlib.machinery import SourceFileLoader
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import RobertaModel
from fairseq.models.roberta import XLMRModel
from src.model.bilstm import BiLSTM
from src.resources import hparams
from torch import nn
import os